In [ ]:
!pip install transformers==4.28.0
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
!pip install datasets

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("")

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel,AutoTokenizer, GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

def process_csv(file_path):
    df = pd.read_csv(file_path)
    qa_pairs = []

    for index, row in df.iterrows():
        question = row['question']
        answer = row['answer']
        qa_pairs.append(f"Question: {question}\nAnswer: {answer}\n")

    return qa_pairs

def load_dataset(file_path, tokenizer):
    qa_pairs = process_csv(file_path)
    tokenized_dataset = tokenizer(qa_pairs, truncation=True,
                                  padding='max_length', max_length=128,
                                  return_tensors="pt")
    dataset = Dataset.from_dict(tokenized_dataset)
    return dataset

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token


# Load and preprocess the dataset
train_dataset = load_dataset("/content/question.csv", tokenizer)
valid_dataset = load_dataset("/content/valid.csv", tokenizer)

# Configure and train the model using the Trainer class
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=100,
    warmup_steps=0,
    logging_dir="logs",
    evaluation_strategy="steps",
    save_total_limit=3,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

trainer.train()


In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_mentalhealth_counselling_gpt2")


In [ ]:

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_mentalhealth_counselling_gpt2")

def ask_question(question, model, tokenizer, max_length=128, num_return_sequences=1):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()

    # Truncate the answer after the first newline character
    answer = answer.split("\n")[0]

    return answer


# Ask questions using the fine-tuned model
question = "Every winter I find myself getting sad because of the weather. How can I fight this?"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

question = "I'm constantly in a bad mood and I have no energy. Is that depression?"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

question = "I just feel sad all the time and I don't like anyone in my family. I feel like they're trying to control me and won't let me grow."
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")


In [ ]:

question = "I live a normal life. I have tons of friends and family, but I feel lonely."
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

In [ ]:
question = "They don't go away, and I feel like I'm going crazy. Does that ever stop? Can it be a symptom of medication?"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

In [ ]:
question = "I have a lot on my mind, but all I want to do is stay locked in my room and not socialize with anyone. Why do I feel so alone?"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

In [ ]:
!pip install pyTelegramBotAPI
!pip install openai==0.28

In [ ]:
import telebot
import os
import openai

TelegramBOT_TOKEN = ''
#Add API here
openai.api_key = "sk-"



bot = telebot.TeleBot(TelegramBOT_TOKEN)
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "Welcome to the mental health therapy bot")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    print(message)
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=message.text,
    temperature=0.19,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    bot.reply_to(message, response.choices[0].text)
bot.polling()
